In [1]:
import pandas as pd

In [2]:
import pandas as pd
from pathlib import Path

def trim(f):
    df = pd.read_parquet(f)
    cols = ["model_updates", "batch_size", "damper", "epochs"]
    out_df = df[cols].copy()
    seed = int(str(f.name).split("-")[0])
    out_df["seed"] = seed
    return out_df

In [3]:
train_dir = Path("2020-05-01") / "train"
train_dfs = list(train_dir.glob("*.parquet"))
out_dfs = []
for k, train_df in enumerate(train_dfs):
    if k % 100 == 0:
        print(k / len(train_dfs))
    out_df = trim(train_df)
    out_dfs.append(out_df)

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


In [4]:
sum(out_df.memory_usage().sum() for out_df in out_dfs) / 1024**2

849.2406463623047

In [14]:
out_df = pd.concat(out_dfs)

In [15]:
out_df.dtypes

model_updates      int64
batch_size         int64
damper            object
epochs           float64
seed               int64
dtype: object

In [16]:
out_df.describe()

,model_updates,batch_size,epochs,seed
count,2.225913e+07,2.225913e+07,2.225913e+07,2.225913e+07
mean,1.369796e+04,4.062910e+02,6.310809e+01,1.099498e+03
std,9.922940e+03,5.550593e+02,4.719939e+01,5.773160e+01
min,1.000000e+00,3.200000e+01,5.333333e-04,1.000000e+03
25%,5.565000e+03,2.560000e+02,1.693013e+01,1.050000e+03
50%,1.113000e+04,2.560000e+02,5.896533e+01,1.099000e+03
75%,2.133900e+04,2.560000e+02,1.043518e+02,1.149000e+03
max,3.525000e+04,4.096000e+03,1.510033e+02,1.199000e+03


In [17]:
out_df.memory_usage().sum() / 1024**2

1018.9422912597656

In [18]:
dtypes = {
    "model_updates": "uint16",
    "batch_size": "uint16",
    "epochs": "float32",
    "seed": "int16",
}
for c, dtype in dtypes.items():
    out_df[c] = out_df[c].astype(dtype)

In [19]:
out_df.head()

,model_updates,batch_size,damper,epochs,seed
0,1,256,adagrad,0.004267,1096
1,2,256,adagrad,0.008533,1096
2,3,256,adagrad,0.012800,1096
3,4,256,adagrad,0.017067,1096
4,5,256,adagrad,0.021333,1096


In [20]:
out_df.memory_usage().sum() / 1024**2

551.927074432373

In [21]:
out_df.to_parquet("train-df-squashed.parquet", engine="pyarrow", compression="zstd", compression_level=10)

In [22]:
out_df.describe()

,model_updates,batch_size,epochs,seed
count,2.225913e+07,2.225913e+07,2.225913e+07,2.225913e+07
mean,1.369796e+04,4.062910e+02,6.310810e+01,1.099498e+03
std,9.922940e+03,5.550593e+02,4.719939e+01,5.773160e+01
min,1.000000e+00,3.200000e+01,5.333333e-04,1.000000e+03
25%,5.565000e+03,2.560000e+02,1.693013e+01,1.050000e+03
50%,1.113000e+04,2.560000e+02,5.896533e+01,1.099000e+03
75%,2.133900e+04,2.560000e+02,1.043518e+02,1.149000e+03
max,3.525000e+04,4.096000e+03,1.510033e+02,1.199000e+03
